This is just 1 plate per experiment, 3 different dates.

ABCD etc are different.

Column 1 2 3 = TS121 
Column 4 5 6 = TS163 

Column 7 8 9 = TS121 
Column 10 11 12 = TS163 

For example:

A1 A2 A3 are the same

No drugs in the plates.

Plate content (Black Flat BioGreiner Plate)

Col 1-6 -Row A - H:
0, 1E2, 1E3, 1E4, 1E5, 1E6, 1E7, 1E8 nM Arabinose

Col 7-12 - Row A - H: 
0, 1.5E4, 3E4, 6E4, 1.25E5, 2.5E5, 5E5, 1E6 nM Arabinose
 

Files
All files represent the same biological experiment

NB_OD600_mEGFP = old format 
R98 & R97 = new format that will be used in the future

In [1]:
import os
def getSubFolders(folderPath):
    subFolders = []
    for root, dirs, files in os.walk(folderPath):
        for fn in files:            
            if fn.endswith('.csv') and '_stats' in fn:
                pth = os.path.join(root, fn)
                if root in subFolders:
                    continue
                subFolders.append(pth)
    return subFolders

In [2]:
folder_input = '/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/dataset_od_gfp/'
lf = getSubFolders(folder_input)
import os
g = dict()
for subFolder in lf:
    #print subFolder
    plate_name = os.path.split(subFolder)[1]
    g[plate_name]= subFolder


In [3]:
def flatten_l(l0,l1):
    l0.extend(l1)
    return l0

def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]
    
def link_dict(dict1, dict2):    
    result = {key: flatten_l(as_list(dict1.get(key, 0)),as_list(dict2.get(key, 0)))
              for key in set(dict1) | set(dict2)}
    return result

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def mean_std_dict(dict0):
    import numpy as np
    result_m = {key+"_mean": np.mean(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    result_std = {key+"_std": np.std(np.asarray(dict0.get(key,0))) for key in set(dict0)}
    return merge_two_dicts(result_m,result_std)

def getMeanStd(dic_growth):
    if len(dic_growth) > 1:
        dic_comb = dic_growth[0]
        for i in range(1,len(dic_growth)):                        
            dic_comb = link_dict(dic_growth[i], dic_comb)            
    else:
        print "only one value array, no mean and std"
        return dic_growth    
    result = mean_std_dict(dic_comb)
    return result



In [12]:
def getGrowthLagdGFP(stats_file):
    import csv
    m_growth = dict()
    m_lag = dict()
    with open(stats_file) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                #print "row_0",row
                line_count += 1
            else:
                #print row
                line_count += 1
                m_growth[row[0]] = float(row[1])
                m_lag[row[0]] = float(row[2])
                m_mean_exp_dGFPdtOD = float(row[3])
                m_mean_sta_dGFPdtOD = float(row[4])
                m_mean_max_dGFPdtOD = float(row[5])
    return m_growth, m_lag,m_mean_exp_dGFPdtOD,m_mean_sta_dGFPdtOD,m_mean_max_dGFPdtOD


In [14]:
growth_rate = dict()
lag_phase = dict()

all_file = []
growth_all = []
lag_all = []
mean_exp_dGFPdtOD = []
mean_sta_dGFPdtO = []
mean_max_dGFPdtOD = []

for k_group_ex in g:
    growth_rate[k_group_ex] = dict()
    lag_phase[k_group_ex] = dict()
    
    growth_rate[k_group_ex]['mixted_date'] = dict()
    lag_phase[k_group_ex]['mixted_date'] = dict()

    stats_file_p = g[k_group_ex]    
    growth_rate[stats_file_p]= dict()
    lag_phase[stats_file_p]= dict()        

    all_file.append(stats_file_p)
    dic_g, dic_la,m_mean_exp_dGFPdtOD,m_mean_sta_dGFPdtOD,m_mean_max_dGFPdtOD = getGrowthLagdGFP(stats_file_p)

    growth_all.append(dic_g)
    lag_all.append(dic_la)
    mean_exp_dGFPdtOD.append(m_mean_exp_dGFPdtOD)
    mean_sta_dGFPdtO.append(m_mean_sta_dGFPdtOD)
    mean_max_dGFPdtOD.append(m_mean_max_dGFPdtOD)
    
    
    #growth_rate[stats_file_p] = getMeanStd(dic_growth)
    #lag_phase[stats_file_p] = getMeanStd(dic_lag)
        
    #growth_rate[k_group_ex]['mixted_date'] = getMeanStd(dic_growth_all)
    #lag_phase[k_group_ex]['mixted_date'] = getMeanStd(dic_lag_all)
        
        

In [17]:
def sort_nicely(l):
    import re
    """ Sort the given list in the way that humans expect.
    """
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    l.sort(key=alphanum_key)
    return l



In [19]:
output_path

'/home/xsong/Desktop/Ana-promoter-derivetive/Nadine/dataset/dataset_od_gfp/growth_rate_lag_phase.csv'

In [18]:
import csv
def writeToFile(f,all_plates,h):
    write_head = True
    for key in sorted(all_plates.keys()):
        if write_head:
            f.write("%s"%(key))
        else:
            f.write("%s\n"%(key))
        for k_date in sort_nicely(all_plates[key].keys()):
            string_to_write = "\t"
            head = "\t"
            for k_well in sort_nicely(all_plates[key][k_date].keys()):
                head = head + k_well+h + "\t"
                string_to_write = string_to_write + str(all_plates[key][k_date][k_well])+"\t"            
            if write_head:
                f.write("%s\n"%(head))
                write_head = False            
            f.write("%s"%(k_date))
            #f.write("%s\n"%(head))
            f.write("%s\n"%(string_to_write))


output_path = os.path.join(folder_input,'growth_rate_lag_phase.csv')
with open(output_path, 'w') as f:
    writeToFile(f,all_plates_growth,"_growth_rate")
    f.write("\n")
    writeToFile(f,all_plates_lagPhase,"_lag_phase")
    
            
    
            